In [26]:
VERSION = "cv-corpus-6.1-2020-12-11"
LANGUAGE = "en"

In [8]:
import librosa
import argparse
import csv
import json
import logging
import multiprocessing
import os
import subprocess
import sys
import tarfile
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import List
import yaml

import sox
from sox import Transformer
from tqdm import tqdm
import pandas as pd
import numpy as np

In [5]:
validated = pd.read_csv("small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv",sep="\t")

array([1016262,  315571,  940842, ..., 1176911,  185601,  637011])

In [27]:
ln = validated.shape[0]
train_ix = np.random.choice(ln,1000,replace=False)
idx2 = validated.index.difference(train_ix)
test_ix = np.random.choice(idx2,200,replace=False)
idx3 = validated.index.difference(np.concatenate([test_ix,train_ix]))
val_ix = np.random.choice(idx3,100,replace=False)


In [31]:
validated['path'].loc[train_ix].to_list()

['common_voice_en_18446803.mp3',
 'common_voice_en_118293.mp3',
 'common_voice_en_18680774.mp3',
 'common_voice_en_18148664.mp3',
 'common_voice_en_19653888.mp3',
 'common_voice_en_19801158.mp3',
 'common_voice_en_18733056.mp3',
 'common_voice_en_22757364.mp3',
 'common_voice_en_76526.mp3',
 'common_voice_en_18713984.mp3',
 'common_voice_en_206746.mp3',
 'common_voice_en_17727459.mp3',
 'common_voice_en_17262464.mp3',
 'common_voice_en_17279616.mp3',
 'common_voice_en_665458.mp3',
 'common_voice_en_18006898.mp3',
 'common_voice_en_17577353.mp3',
 'common_voice_en_20219398.mp3',
 'common_voice_en_47424.mp3',
 'common_voice_en_18709346.mp3',
 'common_voice_en_696831.mp3',
 'common_voice_en_21852581.mp3',
 'common_voice_en_674460.mp3',
 'common_voice_en_22276123.mp3',
 'common_voice_en_142335.mp3',
 'common_voice_en_18941920.mp3',
 'common_voice_en_23879244.mp3',
 'common_voice_en_21051586.mp3',
 'common_voice_en_22912101.mp3',
 'common_voice_en_18887380.mp3',
 'common_voice_en_19350600.m

In [33]:


parser = argparse.ArgumentParser(description='Downloads and processes Mozilla Common Voice dataset.')
parser.add_argument("--data_root", default='small/', type=str, help="Directory to store the dataset.")
parser.add_argument('--manifest_dir', default='./', type=str,help='Output directory for manifests')
parser.add_argument("--num_workers", default=multiprocessing.cpu_count(), type=int, help="Workers to process dataset.")
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--n_channels', default=1, type=int, help='Number of channels for output wav files')
parser.add_argument(
    '--files_to_process',
    nargs='+',
    default=['test.tsv', 'dev.tsv', 'train.tsv'],
    type=str,
    help='list of *.csv file names to process',
    required=False
)
parser.add_argument(
    '--version',
    default='cv-corpus-5.1-2020-06-22',
    type=str,
    required=False,
    help='Version of the dataset (obtainable via https://commonvoice.mozilla.org/en/datasets',
)
parser.add_argument(
    '--language',
    default='en',
    type=str,
    required=False,
    help='Which language to download.(default english,'
    'check https://commonvoice.mozilla.org/en/datasets for more language codes',
)
sys.argv = ['-f']
args = parser.parse_args(['--version',VERSION,'--files_to_process','validated.tsv'])

COMMON_VOICE_URL = (
    f"https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/"
    "{}/{}.tar.gz".format(args.version, args.language)
)



def create_manifest(data: List[tuple], output_name: str, manifest_path: str):
    output_file = Path(manifest_path) / output_name
    output_file.parent.mkdir(exist_ok=True, parents=True)

    with output_file.open(mode='w') as f:
        for wav_path, duration, text in tqdm(data, total=len(data)):
            if wav_path:
              f.write(
                  json.dumps({'audio_filepath': os.path.abspath(wav_path), "duration": duration, 'text': text}) + '\n'
              )


def process_files(csv_file, data_root, num_workers):
    """ Read *.csv file description, convert mp3 to wav, process text.
        Save results to data_root.
    Args:
        csv_file: str, path to *.csv file with data description, usually start from 'cv-'
        data_root: str, path to dir to save results; wav/ dir will be created
    """
    wav_dir = os.path.join(data_root, 'wav/')
    os.makedirs(wav_dir, exist_ok=True)
    audio_clips_path = "/Users/dami.osoba/Downloads/cv-corpus-6.1-2020-12-11/en/clips/"


    def process(x):
        file_path, text = x
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        text = text.lower().strip()
        audio_path = os.path.join(audio_clips_path, file_path)
        output_wav_path = os.path.join(wav_dir, file_name + '.wav')
        tfm = Transformer()
        tfm.rate(samplerate=args.sample_rate)
        tfm.channels(n_channels=args.n_channels)
        # tfm.build(input_filepath=audio_path, output_filepath=output_wav_path)
        os.system(
        f'ffmpeg -i {audio_path} -ar {args.sample_rate} {output_wav_path} -y'
    )
        print(output_wav_path)
        duration = librosa.get_duration(filename=output_wav_path)
        return output_wav_path, duration, text


    logging.info('Converting mp3 to wav for {}.'.format(csv_file))
    with open(csv_file) as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        next(reader, None)  # skip the headers
#         curated = os.listdir(f'{audio_clips_path}')
        curated = validated['path'].loc[test_ix].to_list()
        data = [(row['path'], row['sentence']) for row in reader if row['path'] in curated] 
        with ThreadPool(num_workers) as pool:
            data = list(tqdm(pool.imap(process, data), total=len(data)))
    return data


def main():
    data_root = args.data_root
    os.makedirs(data_root, exist_ok=True)

    target_unpacked_dir = os.path.join(data_root, "CV_unpacked")

    if os.path.exists(target_unpacked_dir):
        logging.info('Find existing folder {}'.format(target_unpacked_dir))
    else:
        logging.info("Could not find Common Voice, Downloading corpus...")

        commands = [
            'wget',
            '--user-agent',
            '"Mozilla/5.0 (Windows NT 10.0; WOW64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"',
            '-P',
            data_root,
            f'{COMMON_VOICE_URL}',
        ]
        commands = " ".join(commands)
        subprocess.run(commands, shell=True, stderr=sys.stderr, stdout=sys.stdout, capture_output=False)
        filename = f"{args.language}.tar.gz"
        target_file = os.path.join(data_root, os.path.basename(filename))

        os.makedirs(target_unpacked_dir, exist_ok=True)
        logging.info("Unpacking corpus to {} ...".format(target_unpacked_dir))
        tar = tarfile.open(target_file)
        tar.extractall(target_unpacked_dir)
        tar.close()

    folder_path = os.path.join(target_unpacked_dir, args.version + f'/{args.language}/')

    for csv_file in args.files_to_process:
        data = process_files(
            csv_file=os.path.join(folder_path, csv_file),
            data_root=os.path.join(data_root, 'test'),
            num_workers=args.num_workers,
        )
        logging.info('Creating manifests...')
        create_manifest(
            data=data,
#             output_name=f'commonvoice_{os.path.splitext(csv_file)[0]}_manifest.json',
            output_name=f'commonvoice_test_manifest.json',

            manifest_path=args.manifest_dir,
        )


if __name__ == "__main__":
    main()


  2%|▏         | 3/200 [00:00<00:14, 13.34it/s]

small/test/wav/common_voice_en_20275923.wav
small/test/wav/common_voice_en_505749.wav
small/test/wav/common_voice_en_20390368.wav
small/test/wav/common_voice_en_17390894.wav


  4%|▍         | 8/200 [00:00<00:10, 18.09it/s]

small/test/wav/common_voice_en_19698716.wav
small/test/wav/common_voice_en_691734.wav
small/test/wav/common_voice_en_19845624.wav
small/test/wav/common_voice_en_18050947.wav
small/test/wav/common_voice_en_72985.wav


  6%|▋         | 13/200 [00:00<00:09, 19.11it/s]

...

 98%|█████████▊| 195/200 [00:09<00:00, 19.99it/s]

small/test/wav/common_voice_en_20472912.wav
small/test/wav/common_voice_en_21130256.wav
small/test/wav/common_voice_en_21143201.wav
small/test/wav/common_voice_en_21328728.wav


100%|██████████| 200/200 [00:10<00:00, 19.53it/s]

small/test/wav/common_voice_en_21333738.wav
small/test/wav/common_voice_en_21343030.wav
small/test/wav/common_voice_en_21867270.wav
small/test/wav/common_voice_en_22924255.wav
small/test/wav/common_voice_en_23593247.wav



100%|██████████| 200/200 [00:00<00:00, 19470.81it/s]
